<a href="https://colab.research.google.com/github/vfeng02/wav2vec_cantonese_xl/blob/main/wav2vec_xl_fleurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.24.0
!pip install jiwer
!pip install torchaudio
!pip install librosa

In [ ]:
from datasets import load_dataset, load_metric, Audio

fleurs = load_dataset("google/xtreme_s", "fleurs.yue_hant_hk")

In [ ]:
fleurs = fleurs.remove_columns(["id", "num_samples", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:"\“\%\‘\”\�\．\⋯\！\－\：\–\。\》\,\）\,\？\；\～\~\…\︰\，\（\」\‧\《\﹔\、\—\／\,\「\﹖\·\' a-zA-Z]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

fleurs = fleurs.map(remove_special_characters)

In [ ]:
show_random_elements(fleurs["train"].remove_columns(["path","audio"]))

In [ ]:
phones_column_train = [""] * len(fleurs["train"])
fleurs["train"] = fleurs["train"].add_column("phones", phones_column_train)

phones_column_val = [""] * len(fleurs["validation"])
fleurs["validation"] = fleurs["validation"].add_column("phones", phones_column_val)

phones_column_test = [""] * len(fleurs["test"])
fleurs["test"] = fleurs["test"].add_column("phones", phones_column_test)

from transformers import T5ForConditionalGeneration, AutoTokenizer

# model = T5ForConditionalGeneration.from_pretrained('/scratch/network/vyfeng/models/g2p_multilingual_byT5_tiny_16_layers_100')
# tokenizer = AutoTokenizer.from_pretrained('/scratch/network/vyfeng/models/byt5-small')

model = T5ForConditionalGeneration.from_pretrained('charsiu/g2p_multilingual_byT5_tiny_16_layers_100')
tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

def get_phones(batch):
    words = ['<yue>: '+i for i in batch["transcription"]]
    out = tokenizer(words,padding=True,add_special_tokens=False,return_tensors='pt')

    preds = model.generate(**out,num_beams=1,max_length=50)
    phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
    batch["phones"] = (" ").join(phones)
    return batch

fleurs = fleurs.map(get_phones)

In [ ]:
from google.colab import drive
from torch.utils.data import Dataset
drive.mount('/content/drive')

# fleurs_pd = Dataset.to_pandas(fleurs)

path = '/content/drive/My Drive/wav2vec_fleurs/fleurs.csv'
# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   fleurs_pd.to_csv(f)
fleurs.save_to_disk(path)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["phones"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = fleurs.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fleurs["train"].column_names)

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

vocab_dict

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

In [ ]:
import json
with open('/content/drive/My Drive/wav2vec_fleurs/fleurs_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer('/content/drive/My Drive/wav2vec_fleurs/fleurs_vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
import huggingface_hub

repo_name = "wav2vec2-xl-yue-fleurs"
tokenizer.push_to_hub(repo_name)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phones"]).input_ids
    return batch

In [ ]:
from datasets import concatenate_datasets

fleurs_train = fleurs["train"].map(prepare_dataset, remove_columns=fleurs["train"].column_names, num_proc=4)
fleurs_val = fleurs["validation"].map(prepare_dataset, remove_columns=fleurs["validation"].column_names, num_proc=4)
fleurs_test = fleurs["test"].map(prepare_dataset, remove_columns=fleurs["test"].column_names, num_proc=4)

fleurs_train = concatenate_datasets([fleurs_train, fleurs_val])


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
  
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
import numpy as np 

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer, "wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-xl-yue-fleurs",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=fleurs_train,
    eval_dataset=fleurs_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

# Preprocessing dataset
Load common-voice data set, resample audio to 16Hz, add